In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import pylab as py	

In [5]:
main_df = pd.read_csv('Bias_correction_ucl.csv')
main_df.sample(5)

,station,Date,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,...,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
285,11.0,2013-07-11,25.9,24.5,84.183594,94.588783,25.493900,23.203183,7.985504,43.063029,...,2.398282,4.213698,0.869906,37.5372,127.085,28.7000,0.6233,5773.572754,25.4,23.7
7323,24.0,2017-08-13,28.7,25.5,61.201973,75.511261,26.858758,23.288472,10.217053,49.123382,...,0.000000,0.000000,0.007756,37.5237,126.909,17.2956,0.2223,5047.767578,27.6,24.0
2185,11.0,2014-07-25,34.4,22.1,53.896526,86.164993,27.338995,23.990024,16.201483,96.824559,...,0.245914,0.000000,0.000000,37.5372,127.085,28.7000,0.6233,5555.624512,28.2,21.6
202,3.0,2013-07-08,26.1,23.2,72.960640,95.537552,28.342721,23.582054,8.952834,23.765362,...,0.000000,0.020585,0.000000,37.5776,127.058,33.3068,0.2661,5804.364746,27.2,23.4
1494,20.0,2013-08-28,30.4,22.1,73.714874,96.804031,26.235513,22.913159,13.912146,40.663272,...,0.210967,7.667776,0.000000,37.6181,127.004,146.5540,4.7296,4632.916992,26.8,22.4


In [6]:
cols = main_df.columns
cols

Index(['station', 'Date', 'Present_Tmax', 'Present_Tmin', 'LDAPS_RHmin',
       'LDAPS_RHmax', 'LDAPS_Tmax_lapse', 'LDAPS_Tmin_lapse', 'LDAPS_WS',
       'LDAPS_LH', 'LDAPS_CC1', 'LDAPS_CC2', 'LDAPS_CC3', 'LDAPS_CC4',
       'LDAPS_PPT1', 'LDAPS_PPT2', 'LDAPS_PPT3', 'LDAPS_PPT4', 'lat', 'lon',
       'DEM', 'Slope', 'Solar radiation', 'Next_Tmax', 'Next_Tmin'],
      dtype='object')

In [7]:
main_df.drop( columns= ['station', 'Date', 'LDAPS_CC2', 'LDAPS_CC3', 'LDAPS_CC4', 'LDAPS_PPT1', 'LDAPS_PPT2', 'LDAPS_PPT3', 'LDAPS_PPT4', 'lat', 'lon'] ,inplace=True)

In [9]:
main_df.rename(columns = {'Present_Tmax': 'PTmax', 
                    'Present_Tmin': 'PTmin', 
                     'LDAPS_RHmin': 'RHmin', # relative humidity min
                     'LDAPS_RHmax': 'RHmax', # max
                     'LDAPS_Tmax_lapse': 'TmaxL', # max lapse temp
                    'LDAPS_Tmin_lapse': 'TminL',
                     'LDAPS_WS': 'WS', 
                     'LDAPS_LH': 'LH',
                     'LDAPS_CC1': 'CC'}, inplace=True)

In [10]:
main_df.columns

Index(['PTmax', 'PTmin', 'RHmin', 'RHmax', 'TmaxL', 'TminL', 'WS', 'LH', 'CC',
       'DEM', 'Slope', 'Solar radiation', 'Next_Tmax', 'Next_Tmin'],
      dtype='object')

In [11]:
inputList = ['PTmax', 'PTmin', 'RHmin', 'RHmax', 'TmaxL', 'TminL', 'WS', 'LH', 'CC','DEM', 'Slope', 'Solar radiation']
input = main_df[inputList]
output = main_df[['Next_Tmax']]

In [15]:
input.isnull().sum()

PTmax              70
PTmin              70
RHmin              75
RHmax              75
TmaxL              75
TminL              75
WS                 75
LH                 75
CC                 75
DEM                 0
Slope               0
Solar radiation     0
dtype: int64

In [16]:
input.shape

(7752, 12)

In [18]:
output

,Next_Tmax
0,29.1
1,30.5
2,31.1
3,31.7
4,31.2
...,...
7747,28.3
7748,28.6
7749,27.8
7750,17.4


In [21]:
output = output.fillna(output.median())
output.shape
output.duplicated().sum()

Next_Tmax    0
dtype: int64

In [22]:
input[['PTmax', 'PTmin', 'RHmin', 'RHmax', 'TmaxL', 'TminL', 'WS', 'LH', 'CC', 'DEM', 'Slope', 'Solar radiation']].describe()

,PTmax,PTmin,RHmin,RHmax,TmaxL,TminL,WS,LH,CC,DEM,Slope,Solar radiation
count,7682.000000,7682.000000,7677.000000,7677.000000,7677.000000,7677.000000,7677.000000,7677.000000,7677.000000,7752.000000,7752.000000,7752.000000
mean,29.768211,23.225059,56.759372,88.374804,29.613447,23.512589,7.097875,62.505019,0.368774,61.867972,1.257048,5341.502803
std,2.969999,2.413961,14.668111,7.192004,2.947191,2.345347,2.183836,33.730589,0.262458,54.279780,1.370444,429.158867
min,20.000000,11.300000,19.794666,58.936283,17.624954,14.272646,2.882580,-13.603212,0.000000,12.370000,0.098475,4329.520508
25%,27.800000,21.700000,45.963543,84.222862,27.673499,22.089739,5.678705,37.266753,0.146654,28.700000,0.271300,4999.018555
50%,29.900000,23.400000,55.039024,89.793480,29.703426,23.760199,6.547470,56.865482,0.315697,45.716000,0.618000,5436.345215
75%,32.000000,24.900000,67.190056,93.743629,31.710450,25.152909,8.032276,84.223616,0.575489,59.832400,1.767800,5728.316406
max,37.600000,29.900000,98.524734,100.000153,38.542255,29.619342,21.857621,213.414006,0.967277,212.335000,5.178230,5992.895996


In [23]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression

In [24]:
x_train, x_test, y_train, y_test= train_test_split(input,output, test_size=0.3, random_state=0)

In [27]:
mv = np.nanmedian(y_train)
y_train_filled = np.nan_to_num(y_train, nan=mv)

In [28]:
y_train_filled.size

5426

In [29]:
mvX = np.nanmedian(x_train)
x_train_filled = np.nan_to_num(x_train, nan=mvX)

In [30]:
Lr = LinearRegression()
Lr.fit(x_train_filled, y_train_filled)

LinearRegression()

In [31]:
interc = Lr.intercept_
slo = Lr.coef_
print('The intercept:', interc, 'and the coeff is: ', slo)

The intercept: [3.65809918] and the coeff is:  [[ 1.51948913e-01  6.36397360e-03 -4.86106280e-03 -4.51046782e-03
   6.91245455e-01  1.02777736e-01 -1.54522231e-01  1.05478571e-02
   2.48481732e-01 -4.51882787e-03  2.42975138e-01  1.34721587e-06]]
